In [ ]:
"""
PYTHON NOTEBOOK STRUCTURE

1 - GET UNIQUE SYMBOLS FROM OUR LIST OF DATA. WE WILL PROVIDE A CSV FILE FOR THIS, SO file_path MUST BE MODIFIED.

2 - USED TO GET FILINGS DATA FOR EACH SYMBOL AND SAVE IT INTO ONE CSV FILE. 
SO, YOU MUST MODIFY formType in query AND ALSO MODIFY THE OUTPUT FILENAME 
OF THE CSV FILE THAT WILL BE SAVED ON YOUR SERVER.
FORM TYPES IMPT TO US ARE (DIVIDED BY COMMAS): S-1, RW, RW WD, EFFECT

3 - USED TO COMBINE (BY TICKER) ALL CSV FILES INTO ONE CSV FILE. 
MODIFICATIONS THAT SHOULD BE MADE IS THE DIRECTORY OF EACH CSV FILE.

4 - SORT THE CSV FILE BY TICKER AND FILEDAT (DATETIME). 
MODIFICATIONS THAT SHOULD BE MADE IS THE CSV FILE THAT WILL BE READ AND OUTPUT FILE NAME


"""

In [ ]:
##symbols and date path: "C:/Users/pcuser/Desktop/EMBER HABITS AND TRADING/samchak impt/fade the gap s1/symbols.xlsx"

import pandas as pd

#MODIFICATION TO BE MADE: change file_path to the path of the excel file with the tickers or symbols. 
# Read the Excel file
file_path = "C:/Users/pcuser/Desktop/EMBER HABITS AND TRADING/samchak impt/fade the gap s1/symbols.xlsx"
df = pd.read_excel(file_path)

# Extract symbols from the first column
symbols = df.iloc[:, 0].tolist()

# Remove duplicate symbols
unique_symbols = list(set(symbols))

# Sort the unique symbols alphabetically
unique_symbols.sort()

# Print the unique symbols
print(unique_symbols)

In [ ]:
from sec_api import QueryApi
import pandas as pd
import json
import time  # Import the time module

queryApi = QueryApi(api_key='API_KEY_HERE')

print (unique_symbols)

print("Unique symbols printed")

# Initialize an empty list to store all responses
all_responses = []

# Set the time between requests (in seconds)
time_between_requests = 1  # Adjust this value as needed


###MODIFICATIONS TO BE MADE: change: formType:\"S-1\"" to the form type you want to get 
for symbol in unique_symbols:
    # Construct the query for the current symbol
    query = {
        "query": {
            "query_string": {
                "query": f"ticker:{symbol} AND formType:\"S-1\""
            }
        },
        "from": "0",
        "size": "200",
        "sort": [{"filedAt": {"order": "desc"}}]
    }
    
    # Get filings for the current symbol
    response = queryApi.get_filings(query)
    
    #print
    print("Got filings for {symbol}!")
    
    # Append the response to the list
    all_responses.extend(response['filings'])  # Ensure you are appending the 'filings' part of the response
    
    # Introduce a time delay between requests
    time.sleep(time_between_requests)

# Convert JSON data to DataFrame
df = pd.json_normalize(all_responses)


#####MODICATIONS TO BE MADE: change the name of the CSV file for each filing type. 
# Save the DataFrame to a CSV file
df.to_csv('combined_s1_filings.csv', index=False)

print("Combined data saved to csv file!")




In [5]:
#combine all the csv files in one csv file

import os
import glob
import pandas as pd
path="C:/Users/pcuser/Documents/GitHub/sam_ember_c9research/fade-the-gap-research/s1_data_csvs_final/"
all_files = glob.glob('path/**/*.csv',recursive=True)


#store each csv file in a dataframe (MODIFICATIONS HERE)

df_1_s1 = pd.read_csv("C:/Users/pcuser/Documents/GitHub/sam_ember_c9research/fade-the-gap-research/s1_data_csvs_final/combined_s1_filings.csv")
df_2_effect = pd.read_csv("C:/Users/pcuser/Documents/GitHub/sam_ember_c9research/fade-the-gap-research/s1_data_csvs_final/combined_effect_filings.csv")
df_3_rw = pd.read_csv("C:/Users/pcuser/Documents/GitHub/sam_ember_c9research/fade-the-gap-research/s1_data_csvs_final/combined_rw_filings.csv")

print ("Putting csv files into dataframes done!")

#set ticker as the index
#df_1_s1.set_index("ticker", inplace = True)
#df_2_effect.set_index("ticker", inplace = True)
#df_3_rw.set_index("ticker", inplace = True)
#df_4_rw_wd.set_index("ticker", inplace = True)

print("Setting ticker column as index for each df done!")
print(df_1_s1)

outer_merged_1 = pd.merge(df_1_s1, df_2_effect, how='outer')
outer_merged_2 = pd.merge(outer_merged_1, df_3_rw, how='outer')

print (outer_merged_2)


outer_merged_2.reset_index()

#MODIFY FILENAME AS NEEDED
outer_merged_2.to_csv('ALLS1DATA.csv', index = False)
print("Merged DFs to CSV file done!")



Putting csv files into dataframes done!
Setting ticker column as index for each df done!
     ticker formType           accessionNo      cik  \
0      AADI    S-1/A  0001193125-17-200203  1422142   
1      AADI    S-1/A  0001193125-17-178180  1422142   
2      AADI      S-1  0001193125-17-124762  1422142   
3      AAGR      S-1  0001213900-23-100019  1848898   
4      AAGR    S-1/A  0001213900-22-045038  1884334   
...     ...      ...                   ...      ...   
5608   ZVSA    S-1/A  0001213900-21-048448  1859007   
5609   ZVSA    S-1/A  0001213900-21-038935  1859007   
5610   ZVSA    S-1/A  0001213900-21-035978  1859007   
5611   ZVSA    S-1/A  0001213900-21-033318  1859007   
5612   ZVSA      S-1  0001213900-21-025890  1859007   

                                companyNameLong  \
0          Aerpio Pharmaceuticals, Inc. (Filer)   
1          Aerpio Pharmaceuticals, Inc. (Filer)   
2          Aerpio Pharmaceuticals, Inc. (Filer)   
3     African Agriculture Holdings Inc. (Filer

      ticker formType           accessionNo      cik  \
0       AADI   EFFECT  9999999995-17-001647  1422142   
1       AADI   EFFECT  9999999995-18-000803  1422142   
2       AADI   EFFECT  9999999995-18-002399  1422142   
3       AADI   EFFECT  9999999995-19-000195  1422142   
4       AADI   EFFECT  9999999995-21-001458  1422142   
...      ...      ...                   ...      ...   
11061   ZVSA    S-1/A  0001493152-23-036393  1859007   
11062   ZVSA    S-1/A  0001493152-23-037493  1859007   
11063   ZVSA    S-1/A  0001493152-23-042336  1859007   
11064   ZVSA    S-1/A  0001493152-23-043620  1859007   
11065   ZVSA    S-1/A  0001493152-23-043877  1859007   

                            companyNameLong                   companyName  \
0      Aerpio Pharmaceuticals, Inc. (Filer)  Aerpio Pharmaceuticals, Inc.   
1      Aerpio Pharmaceuticals, Inc. (Filer)  Aerpio Pharmaceuticals, Inc.   
2      Aerpio Pharmaceuticals, Inc. (Filer)  Aerpio Pharmaceuticals, Inc.   
3      Aerpio Pharm

Merged DFs to CSV file done!


In [6]:
# Read the CSV file into a pandas DataFrame
df = pd.read_csv("C:/Users/pcuser/Documents/GitHub/sam_ember_c9research/fade-the-gap-research/ALLS1DATA.csv")

# Convert the "filedAt" column to datetime objects
df['filedAt'] = pd.to_datetime(df['filedAt'])

# Sort the DataFrame by "ticker" and then by "filedAt"
sorted_df = df.sort_values(by=['ticker', 'filedAt'])

# Reorder the columns to put "filedAt" in the second column
sorted_df = sorted_df[['ticker', 'filedAt'] + [col for col in sorted_df.columns if col != 'ticker' and col != 'filedAt']]

# Save the sorted DataFrame back to a CSV file if needed
sorted_df.to_csv("ALLS1DATA_sortedbyfilingtime.csv", index=False)

C:\Users\pcuser\AppData\Local\Temp\ipykernel_23784\2063785177.py:5: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['filedAt'] = pd.to_datetime(df['filedAt'])
